#### Generate the distance between intervention and the nearest AED / nearest hospital

In this notebook, we calculate the distance between each intervention geo coordinate and the nearest AED / nearest hospital using pyproj package with parameter `ellps='WGS84'`, which is the ellipsoid of earth shape.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
itv_aed = pd.read_csv(
    '/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/interventions_aed_related_with_province.csv',
    low_memory=False)
aed = pd.read_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/aed_location_latlon.csv')
hospital = pd.read_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/hospital.csv')

In [3]:
itv_aed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43093 entries, 0 to 43092
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mission_id                        43093 non-null  int64  
 1   service_name                      38978 non-null  object 
 2   postalcode_permanence             27284 non-null  float64
 3   cityname_permanence               28483 non-null  object 
 4   streetname_permanence             28631 non-null  object 
 5   housenumber_permanence            2156 non-null   float64
 6   latitude_permanence               39916 non-null  float64
 7   longitude_permanence              40255 non-null  float64
 8   permanence_short_name             43053 non-null  object 
 9   permanence_long_name              38982 non-null  object 
 10  vector_type                       42545 non-null  object 
 11  eventtype_firstcall               27434 non-null  object 
 12  even

In [4]:
aed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15775 entries, 0 to 15774
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            15775 non-null  float64
 1   type          5661 non-null   object 
 2   address       15775 non-null  object 
 3   number        13577 non-null  float64
 4   postal_code   15775 non-null  int64  
 5   municipality  15775 non-null  object 
 6   province      15775 non-null  object 
 7   location      8962 non-null   object 
 8   public        8656 non-null   object 
 9   available     4739 non-null   object 
 10  hours         1148 non-null   object 
 11  full_address  15775 non-null  object 
 12  lat           15775 non-null  float64
 13  lon           15775 non-null  float64
dtypes: float64(4), int64(1), object(9)
memory usage: 1.7+ MB


In [5]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_hospital  117 non-null    object 
 1   province       114 non-null    object 
 2   lat            116 non-null    float64
 3   lon            116 non-null    float64
dtypes: float64(2), object(2)
memory usage: 3.8+ KB


In [6]:
from pyproj import Geod # Calculate distance between two points on the earth.

geod = Geod(ellps='WGS84')

def find_closest_distance(df1, df2): # calculate the distance between aed related intervention and the closest aed device 
    distances = []
    for index1, row1 in df1.iterrows():
        min_distance = float('inf')
        for index2, row2 in df2.iterrows():
            # calculate the distance between two points
            _, _, distance = geod.inv(row1['lon_itv'], row1['lat_itv'], row2['lon'],
                                      row2['lat'])
            # update the min distance
            if distance < min_distance:
                min_distance = distance
        # append the min distance to the list
        distances.append(min_distance)
    return distances



itv_aed['aed_distance'] = find_closest_distance(itv_aed, aed)
itv_aed.head()

,mission_id,service_name,postalcode_permanence,cityname_permanence,streetname_permanence,housenumber_permanence,latitude_permanence,longitude_permanence,permanence_short_name,permanence_long_name,...,t0_Month,t0_DayName,t7_Hour,t7_Day,t7_Month,t7_DayName,province,intervention_time_(t1confirmed),departure_time_(t1confirmed),aed_distance
0,20222490147,FB PDS SCHA [PASI Paul Brien] SIAMU,1030.0,Schaarbeek (Schaarbeek),ChaussÈe de Haecht,NaN,50.86948,4.38649,ABSCHA01A,AMB BRIEN 1,...,9.0,Tuesday,16.0,6.0,9.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,5.954143
1,20222490147,FB PDS SCHA [PASI Paul Brien] SIAMU,1030.0,Schaarbeek (Schaarbeek),ChaussÈe de Haecht,NaN,50.86948,4.38649,ABSCHA02A,AMB BRIEN 2,...,9.0,Tuesday,17.0,6.0,9.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,5.954143
2,20222490155,HB UR BRUX Europe Elisabeth,1180.0,ukkel (ukkel),de frÈlaan,NaN,50.80470,4.36763,ABBRUX09A,AMB ST ELISABETH 9,...,9.0,Tuesday,17.0,6.0,9.0,Tuesday,Provincie Vlaams-Brabant,NaN,NaN,389.414213
3,20222490162,FB PDS BRUX [PASI CitÈ] SIAMU,1000.0,Brussel (Brussel),Vesaliusstraat,NaN,50.85097,4.36411,ABBRUX11A,AMB CITE 2,...,9.0,Tuesday,17.0,6.0,9.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,13.534626
4,20222490200,FB PDS ANDE [PASI Anderlecht] SIAMU,1070.0,Anderlecht,Bergense Steenweg,NaN,50.83254,4.31199,ABANDE03A,AMB AND 3,...,9.0,Tuesday,18.0,6.0,9.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,46.578200


In [7]:
itv_aed['hospital_distance'] = find_closest_distance(itv_aed, hospital)
itv_aed.to_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/intervention_aed_related_distance.csv', index=False)

In [8]:
itv_ca = itv_aed[itv_aed['eventtype_trip'].isin([
    'P003 - Cardiac arrest'
    ])]

itv_ca.to_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/intervention_ca_distance.csv', index=False)